In [ ]:
!nvidia-smi

In [ ]:
import scienceplots
import matplotlib.pyplot as plt
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from tqdm import tqdm
from IPython.display import display
from diffusers.utils import make_image_grid

In [ ]:
fully_trained_model = "CompVis/stable-diffusion-v1-4"
finetuned_model = "rhfeiyang/art-free-diffusion-v1"

prompt = 'Mona Lisa by Leonardo da Vinci'
prompt = 'Starry Night by Van Gogh'

p_d_ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.55, 0.6]
inference_steps = 50
eta = 1.0
guidance_scale = 7.5
seed = 42


plt.style.use(['science', 'notebook', 'grid'])
plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 15,
    'axes.labelsize': 15,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
})

In [ ]:
def plot_denoised_iterations(original_pipeline, finetuned_i2i_pipeline, prompt, eval_infer_timestep_chkpts, 
                             inference_steps=50, eta=0.0, seed=42, guidance_scale=7.5, figsize=(20, 7)):
    num_rows = 2
    num_cols = len(eval_infer_timestep_chkpts)
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=figsize)
    fig.subplots_adjust(hspace=0.5)
    
    for i, eval_infer_chkpt in enumerate(tqdm(eval_infer_timestep_chkpts, desc="Processing checkpoints")):
        noisy_sample = original_pipeline(
            prompt=prompt,
            num_inference_steps=inference_steps,
            denoising_end=eval_infer_chkpt,
            output_type="latent",
            eta=eta,
            generator=torch.manual_seed(seed),
        ).images
        noisy_image = original_pipeline.image_processor.postprocess(noisy_sample, output_type="pil", do_denormalize=([True] * noisy_sample.shape[0]))[0]
        
        axes[0, i].imshow(noisy_image)
        axes[0, i].set_title(f"Denoised {round(eval_infer_chkpt*100, 2)}% of T")
        axes[0, i].axis('off')
        
        denoised_image = finetuned_i2i_pipeline(
            prompt=prompt,
            num_inference_steps=inference_steps,
            guidance_scale=guidance_scale,
            eta=eta,
            generator=torch.manual_seed(seed),
            denoising_start=eval_infer_chkpt,
            image=noisy_sample
        ).images[0]
        
        axes[1, i].imshow(denoised_image)
        axes[1, i].set_title(f"Denoised {round((1-eval_infer_chkpt)*100, 2)}%")
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
def compress_pillow_image(img, output_path, quality=85):
  new_width = 400
  width, height = img.size
  aspect_ratio = width / height
  new_height = int(new_width / aspect_ratio)
  img = img.resize((new_width, new_height))

  img.save(output_path, quality=quality)


def generate_image_grids():
    grid_path=f"./{prompt}-"
    
    images_pipeline = pipeline(prompt=prompt, num_images_per_prompt=25).images
    grid_pipeline = make_image_grid(images_pipeline, rows=5, cols=5)
    
    display(grid_pipeline)
    grid_pipeline.save(f"{grid_path}fully_trained-full_image.png")
    compress_pillow_image(grid_pipeline, f"{grid_path}fully_trained_compressed.png", quality=100)
    
    images_finetune = finetune_pipeline(prompt=prompt, num_images_per_prompt=25).images
    grid_finetune = make_image_grid(images_finetune, rows=5, cols=5)
    grid_finetune.save(f"{grid_path}gold_standard-full_image.png")
    compress_pillow_image(grid_finetune, f"{grid_path}gold_standard_compressed.png", quality=100)
    
    display(grid_finetune)

In [ ]:
pipeline = StableDiffusionPipeline.from_pretrained(
    fully_trained_model, 
    safety_checker=None, 
    torch_dtype=torch.float16, 
    use_safetensors=True, 
    cache_dir='/media/respailab/Volume 2/RespAI-Jupyter-Server/huggingface/hub'
).to("cuda")

finetune_pipeline = StableDiffusionPipeline.from_pretrained(
    finetuned_model, 
    safety_checker=None, 
    torch_dtype=torch.float16, 
    use_safetensors=True, 
    cache_dir='/media/respailab/Volume 2/RespAI-Jupyter-Server/huggingface/hub'
).to("cuda")

i2i_ft_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    finetuned_model,
    torch_dtype=torch.float16,
    safety_checker=None,
    use_safetensors=True,
    cache_dir='/media/respailab/Volume 2/RespAI-Jupyter-Server/huggingface/hub'
).to("cuda")

In [ ]:
plot_denoised_iterations(
    pipeline, 
    i2i_ft_pipe, 
    prompt, 
    p_d_ratios,
    inference_steps=inference_steps, 
    eta=eta, 
    seed=seed, 
    guidance_scale=guidance_scale, 
    figsize=(20, 7)
)

In [ ]:
generate_image_grids()